## Semantic chunker
Now do the same thing using the `Semantic Chunker`. You will store the embeddings in a different table by adding `+"_SEMANTIC"` to the table name. LangChains implementation of the `Semantic Chunker` is based on [Greg Kamradt's work](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb), there you can also find more information on the `Semantic Chunker` and other chunking techniques.

👉 For the semantic chunker we need to install a new package: [langchain-experimental](https://pypi.org/project/langchain-experimental/). Run the following command in your terminal. Make sure to run it inside the virtual environment.

```sh
pip install langchain-experimental
```

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

from langchain_community.vectorstores.hanavector import HanaDB

from hdbcli import dbapi
import configparser

import variables

In [ ]:
# Connect to HANA
config = configparser.ConfigParser()
config.read('.user.ini')
connection = dbapi.connect(
    address=config.get('hana', 'url'), 
    port=config.get('hana', 'port'), 
    user=config.get('hana', 'user'),
    password=config.get('hana', 'passwd'),
    autocommit=True,
    sslValidateCertificate=False
)

In [ ]:
# Load custom documents
loader = PyPDFDirectoryLoader('documents/')
documents = loader.load()

In [ ]:
# embedding instance is used during semantic chunking
embeddings = OpenAIEmbeddings(deployment_id=variables.EMBEDDING_DEPLOYMENT_ID)

In [ ]:
# create semantic text chunks
text_splitter = SemanticChunker(embeddings)
texts = text_splitter.split_documents(documents)
print(f"Number of document chunks: {len(texts)}")

In [ ]:

db = HanaDB(
    embedding=embeddings, connection=connection, table_name=variables.SEMANTIC_EMBEDDING_TABLE
)

# Delete already existing documents from the table
db.delete(filter={})

# add the loaded document chunks
db.add_documents(texts)

## Check the embeddings in SAP HANA Cloud Vector Engine

👉 Go back to [HANA Cloud Database Explorer](https://central-hana-cloud-instance-us-77belmsm.hana-tooling.ingress.orchestration.prod-us10.hanacloud.ondemand.com/hrtt/sap/hana/cst/catalog/cockpit-index.html?target=sqlconsole&databaseid=C2366).

👉 Open a second tab and compare the embedding table with the semantic chunks to the chunks using the standard character text splitter:

```sql
SELECT VEC_TEXT, VEC_META, TO_NVARCHAR(VEC_VECTOR) FROM EMBEDDINGS_CODEJAM_+<YOUR_NAME>+"_SEMANTIC"
```

👉 Go back to exercise [07-RAG](07-RAG.ipynb) and run the RAG code again but this time use the table with the newly created semantic chunks.

[Next exercise](09-use-multimodal-models.ipynb)